--- Day 21: Dirac Dice ---

There's not much to do as you slowly descend to the bottom of the ocean. The submarine computer challenges you to a nice game of Dirac Dice.

This game consists of a single die, two pawns, and a game board with a circular track containing ten spaces marked 1 through 10 clockwise. Each player's starting space is chosen randomly (your puzzle input). Player 1 goes first.

Players take turns moving. On each player's turn, the player rolls the die three times and adds up the results. Then, the player moves their pawn that many times forward around the track (that is, moving clockwise on spaces in order of increasing value, wrapping back around to 1 after 10). So, if a player is on space 7 and they roll 2, 2, and 1, they would move forward 5 times, to spaces 8, 9, 10, 1, and finally stopping on 2.

After each player moves, they increase their score by the value of the space their pawn stopped on. Players' scores start at 0. So, if the first player starts on space 7 and rolls a total of 5, they would stop on space 2 and add 2 to their score (for a total score of 2). The game immediately ends as a win for any player whose score reaches at least 1000.

Since the first game is a practice game, the submarine opens a compartment labeled deterministic dice and a 100-sided die falls out. This die always rolls 1 first, then 2, then 3, and so on up to 100, after which it starts over at 1 again. Play using this die.

For example, given these starting positions:

```
Player 1 starting position: 4
Player 2 starting position: 8
```

This is how the game would go:
```
Player 1 rolls 1+2+3 and moves to space 10 for a total score of 10.
Player 2 rolls 4+5+6 and moves to space 3 for a total score of 3.
Player 1 rolls 7+8+9 and moves to space 4 for a total score of 14.
Player 2 rolls 10+11+12 and moves to space 6 for a total score of 9.
Player 1 rolls 13+14+15 and moves to space 6 for a total score of 20.
Player 2 rolls 16+17+18 and moves to space 7 for a total score of 16.
Player 1 rolls 19+20+21 and moves to space 6 for a total score of 26.
Player 2 rolls 22+23+24 and moves to space 6 for a total score of 22.
...after many turns...

Player 2 rolls 82+83+84 and moves to space 6 for a total score of 742.
Player 1 rolls 85+86+87 and moves to space 4 for a total score of 990.
Player 2 rolls 88+89+90 and moves to space 3 for a total score of 745.
Player 1 rolls 91+92+93 and moves to space 10 for a final score, 1000.
```

Since player 1 has at least 1000 points, player 1 wins and the game ends. At this point, the losing player had 745 points and the die had been rolled a total of 993 times; 745 * 993 = 739785.

Play a practice game using the deterministic 100-sided die. The moment either player wins, what do you get if you multiply the score of the losing player by the number of times the die was rolled during the game?

Your puzzle answer was 752247.

--- Part Two ---

Now that you're warmed up, it's time to play the real game.

A second compartment opens, this time labeled Dirac dice. Out of it falls a single three-sided die.

As you experiment with the die, you feel a little strange. An informational brochure in the compartment explains that this is a quantum die: when you roll it, the universe splits into multiple copies, one copy for each possible outcome of the die. In this case, rolling the die always splits the universe into three copies: one where the outcome of the roll was 1, one where it was 2, and one where it was 3.

The game is played the same as before, although to prevent things from getting too far out of hand, the game now ends when either player's score reaches at least 21.

Using the same starting positions as in the example above, player 1 wins in 444356092776315 universes, while player 2 merely wins in 341960390180808 universes.

Using your given starting positions, determine every possible outcome. Find the player that wins in more universes; in how many universes does that player win?

Your puzzle answer was 221109915584112.

Both parts of this puzzle are complete! They provide two gold stars: **

In [31]:
import itertools
from collections import Counter

In [7]:
def solve1(data):
    positions = [int(it.split()[-1]) for it in data]
    scores = [0, 0]
    
    dice = itertools.cycle(range(1, 101))
    player_turns = itertools.cycle([0, 1])
    
    turn, rolls = 0, 0
    
    while max(scores) < 1000:
        player_idx = next(player_turns)        
        d1 = next(dice)
        d2 = next(dice)
        d3 = next(dice)
        
        new_pos = ((positions[player_idx] + d1 + d2 + d3 - 1) % 10) + 1
        positions[player_idx] = new_pos
        scores[player_idx] += new_pos
        
        if turn < 10:
            print(f'Player {player_idx+1} rolls {d1}+{d2}+{d3} and moves to {new_pos} with score {scores[player_idx]}')
        turn += 1
        rolls += 3
    
    print(f'Losing score: {min(scores)}, rolls: {rolls}, answer: {min(scores) * rolls}')
        
    
# solve1("""
# Player 1 starting position: 4
# Player 2 starting position: 8
# """.strip().splitlines())

solve1("""
Player 1 starting position: 10
Player 2 starting position: 8
""".strip().splitlines())

Player 1 rolls 1+2+3 and moves to 6 with score 6
Player 2 rolls 4+5+6 and moves to 3 with score 3
Player 1 rolls 7+8+9 and moves to 10 with score 16
Player 2 rolls 10+11+12 and moves to 6 with score 9
Player 1 rolls 13+14+15 and moves to 2 with score 18
Player 2 rolls 16+17+18 and moves to 7 with score 16
Player 1 rolls 19+20+21 and moves to 2 with score 20
Player 2 rolls 22+23+24 and moves to 6 with score 22
Player 1 rolls 25+26+27 and moves to 10 with score 30
Player 2 rolls 28+29+30 and moves to 3 with score 25
Losing score: 753, rolls: 999, answer: 752247


In [66]:
cache = {}

WIN = 21


def play(p1_pos, p1_score, p2_pos, p2_score, turn):
    cache_key = (p1_pos, p1_score, p2_pos, p2_score, turn)
    if cache_key in cache:
        return cache[cache_key]
        
    if p1_score >= WIN:
        return [1, 0]
    if p2_score >= WIN:
        return [0, 1]
    
    res = [0, 0]
    pos = p1_pos if turn == 1 else p2_pos
    
    for roll in itertools.product((1,2,3), (1,2,3), (1,2,3)):        
        new_pos = ((pos + sum(roll) - 1) % 10) + 1
        
        if turn == 1:            
            r = play(new_pos, p1_score + new_pos, p2_pos, p2_score, 2)
        else:
            r = play(p1_pos, p1_score, new_pos, p2_score + new_pos, 1)
            
        res[0] += r[0]
        res[1] += r[1]
        
    cache[cache_key] = res        
    return res
        


def solve2(data):
    positions = [int(it.split()[-1]) for it in data]    
    wins = play(positions[0], 0, positions[1], 0, 1)
    print('Wins:', wins, 'Answer:', max(wins))
        
    
# solve2("""
# Player 1 starting position: 4
# Player 2 starting position: 8
# """.strip().splitlines())


solve2("""
Player 1 starting position: 10
Player 2 starting position: 8
""".strip().splitlines())

Wins: [221109915584112, 117096403483545] Answer: 221109915584112
